In [2]:
import os 

In [3]:
%pwd

'e:\\Final_year_project\\End-to-End-Chest-Cancer-Prediction-Implementation-using-MLOps-DVC-CICD\\research'

In [4]:
os.chdir("../")

In [5]:
%pwd

'e:\\Final_year_project\\End-to-End-Chest-Cancer-Prediction-Implementation-using-MLOps-DVC-CICD'

In [6]:
os.environ["MLFLOW_TRACKING_URI"]="https://dagshub.com/channawarshri/End-to-End-Chest-Cancer-Prediction-Implementation-using-MLOps-DVC-CICD.mlflow/"
os.environ["MLFLOW_TRACKING_USERNAME"]=" channawarshri"
os.environ["MLFLOW_TRACKING_PASSWORD"]="6824692c47a35333f9eac5b10041d5c8edbcef0"


In [21]:
import dagshub
dagshub.init(repo_owner='channawarshri', repo_name='End-to-End-Chest-Cancer-Prediction-Implementation-using-MLOps-DVC-CICD', mlflow=True)

import mlflow
with mlflow.start_run():
  mlflow.log_param('parameter name', 'value')
  mlflow.log_metric('metric name', 1)

[2025-02-15 15:24:43,656: INFO: _client: HTTP Request: GET https://dagshub.com/api/v1/repos/channawarshri/End-to-End-Chest-Cancer-Prediction-Implementation-using-MLOps-DVC-CICD "HTTP/1.1 200 OK"]


Initialized MLflow to track repo 
"channawarshri/End-to-End-Chest-Cancer-Prediction-Implementation-using-MLOps-DVC-CICD"

[2025-02-15 15:24:43,698: INFO: helpers: Initialized MLflow to track repo "channawarshri/End-to-End-Chest-Cancer-Prediction-Implementation-using-MLOps-DVC-CICD"]


Repository channawarshri/End-to-End-Chest-Cancer-Prediction-Implementation-using-MLOps-DVC-CICD initialized!

[2025-02-15 15:24:43,703: INFO: helpers: Repository channawarshri/End-to-End-Chest-Cancer-Prediction-Implementation-using-MLOps-DVC-CICD initialized!]
🏃 View run whimsical-colt-269 at: https://dagshub.com/channawarshri/End-to-End-Chest-Cancer-Prediction-Implementation-using-MLOps-DVC-CICD.mlflow/#/experiments/0/runs/44b76f3386bb4396ae9e2483f798c320
🧪 View experiment at: https://dagshub.com/channawarshri/End-to-End-Chest-Cancer-Prediction-Implementation-using-MLOps-DVC-CICD.mlflow/#/experiments/0


In [22]:
import tensorflow as tf
model = tf.keras.models.load_model("artifacts/training/model.h5")



In [23]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class EvaluationConfig:
    path_of_model: Path
    training_data: Path
    all_params: dict
    mlflow_uri: str
    params_image_size: list
    params_batch_size: int

In [24]:
from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml, create_directories, save_json


In [25]:
class ConfigurationManager:
    def __init__(
        self, 
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])

    
    def get_evaluation_config(self) -> EvaluationConfig:
        eval_config = EvaluationConfig(
            path_of_model="artifacts/training/model.h5",
            training_data="artifacts/data_ingestion/Chest-CT-Scan-data",
            mlflow_uri="https://dagshub.com/channawarshri/End-to-End-Chest-Cancer-Prediction-Implementation-using-MLOps-DVC-CICD.mlflow",
            all_params=self.params,
            params_image_size=self.params.IMAGE_SIZE,
            params_batch_size=self.params.BATCH_SIZE
        )
        return eval_config


In [26]:
import tensorflow as tf
from pathlib import Path
import mlflow
import mlflow.keras
from urllib.parse import urlparse


In [27]:
class Evaluation:
    def __init__(self, config: EvaluationConfig):
        self.config = config

    
    def _valid_generator(self):

        datagenerator_kwargs = dict(
            rescale = 1./255,
            validation_split=0.30
        )

        dataflow_kwargs = dict(
            target_size=self.config.params_image_size[:-1],
            batch_size=self.config.params_batch_size,
            interpolation="bilinear"
        )

        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )

        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="validation",
            shuffle=False,
            **dataflow_kwargs
        )


    @staticmethod
    def load_model(path: Path) -> tf.keras.Model:
        return tf.keras.models.load_model(path)
    

    def evaluation(self):
        self.model = self.load_model(self.config.path_of_model)
        self._valid_generator()
        self.score = model.evaluate(self.valid_generator)
        self.save_score()

    def save_score(self):
        scores = {"loss": self.score[0], "accuracy": self.score[1]}
        save_json(path=Path("scores.json"), data=scores)

    
    def log_into_mlflow(self):
        mlflow.set_registry_uri(self.config.mlflow_uri)
        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme
        
        with mlflow.start_run():
            mlflow.log_params(self.config.all_params)
            mlflow.log_metrics(
                {"loss": self.score[0], "accuracy": self.score[1]}
            )
            # Model registry does not work with file store
            if tracking_url_type_store != "file":

                # Register the model
                # There are other ways to use the Model Registry, which depends on the use case,
                # please refer to the doc for more information:
                # https://mlflow.org/docs/latest/model-registry.html#api-workflow
                mlflow.keras.log_model(self.model, "model", registered_model_name="VGG16Model")
            else:
                mlflow.keras.log_model(self.model, "model")

In [28]:
try:
    config = ConfigurationManager()
    eval_config = config.get_evaluation_config()
    evaluation = Evaluation(eval_config)
    evaluation.evaluation()
    evaluation.log_into_mlflow()

except Exception as e:
   raise e

[2025-02-15 15:24:50,265: INFO: common: yaml file: config\config.yaml loaded successfully]


[2025-02-15 15:24:50,284: INFO: common: yaml file: params.yaml loaded successfully]
[2025-02-15 15:24:50,289: INFO: common: created directory at: artifacts]
Found 102 images belonging to 2 classes.
7/7 [==============================] - 23s 3s/step - loss: 3.2456e-05 - accuracy: 1.0000
[2025-02-15 15:25:15,038: INFO: common: json file saved at: scores.json]


2025/02/15 15:25:17 WARNING mlflow.tensorflow: You are saving a TensorFlow Core model or Keras model without a signature. Inference with mlflow.pyfunc.spark_udf() will not work unless the model's pyfunc representation accepts pandas DataFrames as inference inputs.


[2025-02-15 15:25:21,995: WARNING: save: Found untraced functions such as _jit_compiled_convolution_op, _jit_compiled_convolution_op, _jit_compiled_convolution_op, _jit_compiled_convolution_op, _jit_compiled_convolution_op while saving (showing 5 of 14). These functions will not be directly callable after loading.]
INFO:tensorflow:Assets written to: C:\Users\chann\AppData\Local\Temp\tmpddbk96v1\model\data\model\assets
[2025-02-15 15:25:22,740: INFO: builder_impl: Assets written to: C:\Users\chann\AppData\Local\Temp\tmpddbk96v1\model\data\model\assets]


2025/02/15 15:25:38 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
Registered model 'VGG16Model' already exists. Creating a new version of this model...
2025/02/15 15:26:02 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: VGG16Model, version 2
Created version '2' of model 'VGG16Model'.


🏃 View run victorious-mouse-91 at: https://dagshub.com/channawarshri/End-to-End-Chest-Cancer-Prediction-Implementation-using-MLOps-DVC-CICD.mlflow/#/experiments/0/runs/7133b02ea3fa46beb7334bfbe927e4ef
🧪 View experiment at: https://dagshub.com/channawarshri/End-to-End-Chest-Cancer-Prediction-Implementation-using-MLOps-DVC-CICD.mlflow/#/experiments/0
